In [ ]:
!curl http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip -o covid19.zip

In [1]:
!rm -rf *.pkl

In [6]:
!mkdir ./covid-data

In [3]:
!unzip ./covid19.zip

Archive:  ./covid19.zip
  inflating: 210110COVID19MEXICO.csv  


In [38]:
from pandas import read_csv, to_datetime, read_pickle, DataFrame
from glob import glob
from os.path import getctime
from re import findall, search
from os import listdir
from copy import copy
from datetime import datetime

In [5]:
lista = glob("./*COVID19MEXICO.csv")
lista.sort(key=getctime, reverse=True)
csv_file = lista[0]
fecha = findall("(\d{6})COVID19MEXICO.csv", csv_file)[0]

In [9]:
dir = "./covid-data/" + fecha
!mkdir $dir

In [10]:
i = 0
for chunk in read_csv(csv_file, encoding="latin-1", chunksize=200000):
    chunk["FECHA_ACTUALIZACION"] = to_datetime(chunk["FECHA_ACTUALIZACION"])
    chunk["FECHA_INGRESO"] = to_datetime(chunk["FECHA_INGRESO"])
    chunk["FECHA_SINTOMAS"] = to_datetime(chunk["FECHA_SINTOMAS"])
    chunk["FECHA_DEF"] = chunk["FECHA_DEF"].where(chunk["FECHA_DEF"] != "9999-99-99", None)
    chunk["FECHA_DEF"] = to_datetime(chunk["FECHA_DEF"])
    chunk.to_pickle(f"./covid-data/{fecha}/datos_{i:03d}.pkl")
    i += 1

In [14]:
direcs = listdir("./covid-data")
lista = ["./covid-data/" + file for file in direcs if search("^\d{6}$", file)]
lista.sort(key=getctime, reverse=True)
direc = lista[0]
pkl_files = glob(direc + "/*.pkl" )
pkl_files.sort(key=lambda x: int(x[-7:-4]))
fecha = findall("(\d{6})", direc)[0]

In [28]:
df = read_pickle(pkl_files[0])
index = df['CLASIFICACION_FINAL'].isin([1, 2, 3])
con = df[index].groupby(['ENTIDAD_UM', 'FECHA_INGRESO']).count()['ORIGEN']
result = copy(con)

In [32]:
df.groupby(['ENTIDAD_UM', 'FECHA_INGRESO']).count()['ORIGEN']

ENTIDAD_UM  FECHA_INGRESO
1           2020-01-02       3
            2020-01-03       1
            2020-01-05       1
            2020-01-06       4
            2020-01-07       7
                            ..
32          2020-07-07       1
            2020-07-08       1
            2020-07-10       1
            2020-07-11       1
            2020-07-16       1
Name: ORIGEN, Length: 6033, dtype: int64

In [35]:
df.keys()

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [ ]:
pre = DataFrame({})

In [54]:
min_date = datetime(2020, 1, 1, 0, 0, 0)
max_date = datetime.now().replace(hour=0)
max_date = max_date.replace(minute=0).replace(second=0).replace(microsecond=0)

In [59]:
delta_days = (max_date - min_date).days

In [23]:
for file in pkl_files[1:]:
    df = read_pickle(file)
    index = df['CLASIFICACION_FINAL'].isin([1, 2, 3])
    con = df[index].groupby(['ENTIDAD_UM', 'FECHA_INGRESO']).count()
    result.add(con, fill_value=0)

result.to_csv(f"{direc}/confirmados_{fecha}.csv")

ValueError: cannot join with no overlapping index names

In [24]:
result

ENTIDAD_UM  FECHA_INGRESO
1           2020-02-20       1
            2020-03-12       1
            2020-03-14       1
            2020-03-17       1
            2020-03-18       3
                            ..
32          2020-07-07       1
            2020-07-08       1
            2020-07-10       1
            2020-07-11       1
            2020-07-16       1
Name: ORIGEN, Length: 3684, dtype: int64